In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from unidecode import unidecode
import os

pd.options.display.max_columns=None

In [3]:
data_name = "CYCLONES"
# data_name = "SARMIENTO"

pierna = "DER"

In [4]:
path_base = f"../datos/{data_name}/"
print(f"Leyendo archivos de la carpeta: {path_base}")

files = os.listdir(path_base)
print(f"Cantidad de archivos: {len(files)}")

# recorre los jugadores
df_total = []
for filename in files:
    
    path = path_base + filename
    file = pd.ExcelFile(path)
    df_lst = []
    
    # recorre las hojas de cada excel
    for sheet_name in file.sheet_names:
        try:
            if pierna in sheet_name:

                df = pd.read_excel(path, header=1, sheet_name=sheet_name)
                sheet_name = sheet_name.strip()
                print(f"path: {path}; sheet_name: {sheet_name}")

                vars_interes = [x for x in df.columns if len(x.split("_"))==2]
                vars_interes = ["ITEM"] + vars_interes
                df = df[vars_interes]
                df.columns = df.columns.str.lower().str.strip()
                df["player"] = filename.split(".")[1].strip()
                df = df.melt(id_vars=["item", "player"])
                df["var"] = df["variable"].str.split("_").str[0]
                df["replica"] = df["variable"].str.split("_").str[1]

                ix = ["item", "player", "replica"]
                df = df.pivot(
                    index=ix, 
                    columns=["var"], 
                    values=["value"]
                )
                df = df.reset_index()
                df.columns = [x[1] if x[1]!="" else x[0] for x in df.columns]
                df = df.set_index(ix)
                df.columns = [sheet_name.lower()+"_"+x if x not in ix else x for x in df.columns]

                df_lst.append(df)

            elif "SOCIODEPORTIVAS" in sheet_name:
                
                df_socio = pd.read_excel(path, sheet_name=sheet_name)
                print(f"path: {path}; sheet_name: {sheet_name}")

                df_socio.columns = [
                    unidecode(x)
                    .lower()
                    .strip()
                    .replace(" ", "_")
                    .replace(")", "")
                    .replace("(", "") 
                    for x in df_socio.columns
                ]
                df_socio = df_socio.drop(columns=["item"])
                df_socio["player"] = filename.split(".")[1].strip()
                df_socio = df_socio.set_index(["player"])
                df_socio["posicion_de_juego"] = df_socio["posicion_de_juego"].str.lower()
                df_socio["pierna_habil"] = df_socio["pierna_habil"].str.lower()
                df_socio["lesion_rodilla_ultimos_3_anos"] = df_socio["lesion_rodilla_ultimos_3_anos"].str.lower()
                df_socio = df_socio.rename(columns={"tipo_lesion_de_rodilla": "tipo_de_lesion"})
                df_socio["tipo_de_lesion"] = df_socio["tipo_de_lesion"].str.lower()
        
        except Exception as e:
            print(e)            

    df = pd.concat(df_lst, axis=1).join(df_socio)
    df_total.append(df)

Leyendo archivos de la carpeta: ../datos/CYCLONES/
Cantidad de archivos: 21
path: ../datos/CYCLONES/12. JEFFERSON ANGULO.xlsx; sheet_name: ANGULOS_ROD_DER
path: ../datos/CYCLONES/12. JEFFERSON ANGULO.xlsx; sheet_name: MOMENTOS_ROD_DER
path: ../datos/CYCLONES/12. JEFFERSON ANGULO.xlsx; sheet_name: EMG_DER
path: ../datos/CYCLONES/12. JEFFERSON ANGULO.xlsx; sheet_name: HQ_DER
path: ../datos/CYCLONES/12. JEFFERSON ANGULO.xlsx; sheet_name: FRV_DER
path: ../datos/CYCLONES/12. JEFFERSON ANGULO.xlsx; sheet_name: VELOCIDAD_DER
path: ../datos/CYCLONES/12. JEFFERSON ANGULO.xlsx; sheet_name: SOCIODEPORTIVAS
path: ../datos/CYCLONES/9. DIEGO ARNEDO.xlsx; sheet_name: ANGULOS_ROD_DER
path: ../datos/CYCLONES/9. DIEGO ARNEDO.xlsx; sheet_name: MOMENTOS_ROD_DER
path: ../datos/CYCLONES/9. DIEGO ARNEDO.xlsx; sheet_name: EMG_DER
path: ../datos/CYCLONES/9. DIEGO ARNEDO.xlsx; sheet_name: HQ_DER
path: ../datos/CYCLONES/9. DIEGO ARNEDO.xlsx; sheet_name: FRV_DER
path: ../datos/CYCLONES/9. DIEGO ARNEDO.xlsx; sheet

In [5]:
df_total = pd.concat(df_total)
print(df_total.shape)
df_total.head(3)

(6363, 20)


angulos_rod_der_x  angulos_rod_der_y  \
item player           replica                                         
1    JEFFERSON ANGULO 1                48.321899          -4.337573   
                      2                36.976704          -4.768758   
                      3                53.867855           7.628788   

                               angulos_rod_der_z  momentos_rod_der_x  \
item player           replica                                          
1    JEFFERSON ANGULO 1                -8.497101           -0.071115   
                      2                -6.886652            0.055122   
                      3               -21.518278            0.060003   

                               momentos_rod_der_y  momentos_rod_der_z  \
item player           replica                                           
1    JEFFERSON ANGULO 1                 -0.005196            0.056340   
                      2                 -0.013010            0.017787   
                      3                 -0.011600            0.051897   

                               emg_der_bf  emg_der_st  emg_der_vl  emg_der_vm  \
item player           replica                                                   
1    JEFFERSON ANGULO 1          0.153899    0.281727    0.232937    0.494979   
                      2          0.282068    0.293743    0.230157    0.392641   
                      3          0.471769    0.244241    0.071327    0.380602   

                               hq_der_lat  hq_der_med  frv_der_frv  \
item player           replica                                        
1    JEFFERSON ANGULO 1          0.660687    0.569170     4.023742   
                      2          1.225543    0.748122     4.190571   
                      3          6.614168    0.641722     4.518857   

                               velocidad_der_vel  talla_m  peso_kg  \
item player           replica                                        
1    JEFFERSON ANGULO 1                 3.112761     1.75    72.45   
                      2                 3.056547     1.75    72.45   
                      3                 2.899150     1.75    72.45   

                              posicion_de_juego pierna_habil  \
item player           replica                                  
1    JEFFERSON ANGULO 1         defensa lateral      derecha   
                      2         defensa lateral      derecha   
                      3         defensa lateral      derecha   

                              lesion_rodilla_ultimos_3_anos tipo_de_lesion  
item player           replica                                               
1    JEFFERSON ANGULO 1                                  no             no  
                      2                                  no             no  
                      3                                  no             no

In [7]:
# df_total.query("player=='JUAN DAVID RIVERA'")

In [8]:
df_total.isnull().sum()

angulos_rod_der_x                0
angulos_rod_der_y                0
angulos_rod_der_z                0
momentos_rod_der_x               0
momentos_rod_der_y               0
momentos_rod_der_z               0
emg_der_bf                       0
emg_der_st                       0
emg_der_vl                       0
emg_der_vm                       0
hq_der_lat                       0
hq_der_med                       0
frv_der_frv                      0
velocidad_der_vel                0
talla_m                          0
peso_kg                          0
posicion_de_juego                0
pierna_habil                     0
lesion_rodilla_ultimos_3_anos    0
tipo_de_lesion                   0
dtype: int64

In [9]:
df_total.to_csv(f"../datos/{data_name}_PIERNA_{pierna}.csv", sep="|")

---